In [1]:
import os
import sys
import argparse
from pathlib import Path
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import yaml
from google.oauth2.service_account import Credentials
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError

In [2]:
@dataclass
class WorksheetSpec:
    """Specification for a single worksheet."""
    name: str
    columns: List[str]
    protected_columns: List[str] = None
    formulas: Dict[str, str] = None
    auto_id_config: Dict[str, Any] = None
    
    def __post_init__(self):
        if self.protected_columns is None:
            self.protected_columns = []
        if self.formulas is None:
            self.formulas = {}
    
    @classmethod
    def from_yaml(cls, filepath: str) -> 'WorksheetSpec':
        """Load worksheet spec from YAML file."""
        with open(filepath, 'r') as f:
            data = yaml.safe_load(f)
        
        return cls(
            name=data['name'],
            columns=data['columns'],
            protected_columns=data.get('protected_columns', []),
            formulas=data.get('formulas', {}),
            auto_id_config=data.get('auto_id_config')
        )


In [3]:
def column_letter(col_name, columns):
    if col_name not in columns:
        return None
    col_index = columns.index(col_name)
    if col_index > 25:
        raise ValueError(f"found {col_name}, but it is not in columns (A-Z)")
    return chr(65 + col_index)
    

def top_cell(col_name, columns, begin=2):
    letter = column_letter(col_name, columns)
    if letter is not None:
        return f"{letter}{begin}"
    print(f"{col_name} in formulas was not found in columns {columns}.")
    print(f"Skipping implementation of its formula")
    return None

    
def target_column_condition(col_name, columns, end, begin=2):
    # if col_name not in columns:
    #     print(f"{col_name} in formulas was not found in columns {columns}.")
    #     print(f"Skipping implementation of its formula")
    #     return None
    start_cell = top_cell(col_name, columns, begin)
    col_letter = column_letter(col_name, columns)
    if start_cell:
        return f'=ARRAYFORMULA(IF(ISBLANK({start_cell}:{col_letter}{end})),""'


def formula_component(rule, columns, end, begin=2):
    for col_name in columns:
        col_index = columns.index(col_name)
        col_letter = chr(65 + col_index)
        if col_name in rule:
            rule = rule.replace(col_name, f"{col_letter}{begin}:{col_letter}{end}")
    return rule


def array_formula(target, value):
    return [[target + value]]


def update_args(spec, end):
    for col_name, formula in spec.formulas.items():
        tc = top_cell(col_name, spec.columns)
        af = array_formula(target_column_condition(col_name, spec.columns, end),
                           formula_component(formula, spec.columns, end))
        return tc, af

In [4]:
path_to_dancers = "../../../config/workbook_specs/lookup_tables/dancers.yaml"
spec = WorksheetSpec.from_yaml(path_to_dancers)

In [5]:
beg = 2
end = spec.auto_id_config['count'] + 1
beg, end

(2, 51)

In [6]:
spec

WorksheetSpec(name='dancers', columns=['dancer_id', 'first_name', 'last_name', 'full_name', 'email', 'cell'], protected_columns=['dancer_id', 'full_name'], formulas={'full_name': 'first_name&" "&last_name'}, auto_id_config={'column': 'dancer_id', 'prefix': 'dancer_', 'count': 50})

In [9]:
args = update_args(spec, end), True
args

(('D2', [['=ARRAYFORMULA(IF(ISBLANK(D2:D51)),""B2:B51&" "&C2:C51']]), True)

In [ ]:
# Formulas
for col_name, formula in spec.formulas.items():
    col_index = spec.columns.index(col_name)
    col_letter = chr(65 + col_index)
    
    base = formula.lstrip('=')
    array_formula = f'=RRAYFORMULA(IF(ISBLANK({col_letter}2:{col_letter}{count+1}),"",{base}))
    
    sheets_service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{spec.name}!{col_letter}2",
        valueInputOption='USER_ENTERED',
        body={'values': [[array_formula]]}
    ).execute()
    print(f"    ✓ Formula: {col_name}")

In [ ]:
array_formula = f"=

In [ ]:
    
# 1. Authorize and open the spreadsheet (Assumes you have authorized gspread)
# gc = gspread.service_account() 
# spreadsheet = gc.open("Your Spreadsheet Name")

# 2. Select the 'dances' worksheet
worksheet = spreadsheet.worksheet("dances")

# 3. Define the array formula
# The formula calculates minutes * 60 + seconds, and stops at the last row (51)
array_formula = '=ARRAYFORMULA(IF(ISBLANK(E2:E51), "", E2:E51*60+F2:F51))'

# 4. Use update() to set the formula in cell G2
# The formula must be passed as a list of lists: [['formula']]
worksheet.update('G2', [[array_formula]], raw=False)